In [2]:
import sys
import os
import glob
import time
import pandas as pd
import numpy as np

sys.path.append('..')
import constants.constants as const
import constants.file_handler_constants as fh
from constants.restaurant_constants import *

from packages.restaurant.Restaurant import *
from packages.file_handler_package.file_handler import *

In [5]:
# this module is use for merge dataframe of all page in selected province from 'mulProcess_restaurant_scraping_proxy'

# ** select a province by index of 'ALL_REGION_ID_WONGNAI' (manually)
Idx_of_region = 63
cur_region_data = ALL_REGION_ID_WONGNAI[Idx_of_region]

#
cur_province_en = cur_region_data[0]

all_res_scraping_province = glob.glob(os.path.join(fh.STORE_RESTAURANT_SCRAPING, "res_restaurant_scraping", 'res_restaurant_%s' % (cur_province_en), '*.csv'))

res_merge_df = pd.DataFrame()
for cur_page_path in all_res_scraping_province:
    cur_page_df = pd.read_csv(cur_page_path)
    res_merge_df = pd.concat([res_merge_df, cur_page_df])

# remove duplicate restaurant 
res_merge_df.drop_duplicates(subset=['name', 'sub_name'], inplace=True)
# set new index
res_merge_df.set_index(['name', 'sub_name'], inplace=True)

# create directory to store result of merging restaurant by a province
createDirectory(fh.STORE_MERGE_RESTAURANT_SCRAPING, 'res_merge_restaurant')

# save result dataframe to .csv 
# for example: 'res_restaurant_Mukdahan_page_1_44.csv'
res_file_name = 'res_merge_restaurant_%s.csv' % (cur_province_en)
res_path = os.path.join(fh.STORE_MERGE_RESTAURANT_SCRAPING, 'res_merge_restaurant', res_file_name) 
res_merge_df.to_csv(res_path, encoding="utf-8")


Directory res_merge_restaurant created successfully
